In [ ]:
# EU 
# For One Ye

Vorgehnsweise: 
Visualisierung in Pandas Darstellen 
Probe Dashboard mit plotly Dash erstellen 
Dashboar in Jupyter Book ein binden 
-> weiteren Grafiken umsetzen 

Ziele: 
Auswertung auf Basis EU 

Zeitlicher Verlauf gesamt
Abfragen:
    - Summe burned area pro Monat 
    - Fläche verbrannter Wälder insgesamt 
    - Liste besonders betroffener Länder 

Visualisierungen: 
    - Balken-Diagram: Summe burned area pro Monat 
    - Treemap: Verbrannte Fläche pro Vegetationstyp
    - Weltkarte mit betroffener Länder (gefüllt): Farbe Burned Area
    - DE-Karte: Bubles Größe (Burned area)


In [1]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as px

In [2]:
!"C:\Program Files\MongoDB\Server\6.0\bin\mongod.exe" --dbpath "C:\mongodb_data"

{"t":{"$date":"2022-08-28T11:30:51.512+02:00"},"s":"I",  "c":"NETWORK",  "id":4915701, "ctx":"-","msg":"Initialized wire specification","attr":{"spec":{"incomingExternalClient":{"minWireVersion":0,"maxWireVersion":17},"incomingInternalClient":{"minWireVersion":0,"maxWireVersion":17},"outgoing":{"minWireVersion":6,"maxWireVersion":17},"isInternalClient":true}}}
{"t":{"$date":"2022-08-28T11:30:51.518+02:00"},"s":"I",  "c":"CONTROL",  "id":23285,   "ctx":"main","msg":"Automatically disabling TLS 1.0, to force-enable TLS 1.0 specify --sslDisabledProtocols 'none'"}
{"t":{"$date":"2022-08-28T11:30:53.094+02:00"},"s":"I",  "c":"NETWORK",  "id":4648602, "ctx":"main","msg":"Implicit TCP FastOpen in use."}
{"t":{"$date":"2022-08-28T11:30:53.096+02:00"},"s":"I",  "c":"REPL",     "id":5123008, "ctx":"main","msg":"Successfully registered PrimaryOnlyService","attr":{"service":"TenantMigrationDonorService","namespace":"config.tenantMigrationDonors"}}
{"t":{"$date":"2022-08-28T11:30:53.096+02:00"},"s"

In [3]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]
print(mydb.name)

esa_fire


### Query 1: Flächen in Deutschland mit einer verbrannten Fläche größer als 1 km2

In [7]:
# Querys Mongo DB
query2={
     '$and': [
            { 'country' : 'DE' },
            { 'burned_area'    : { '$gt': 1000000 }}
          ]
}
points =mycol.find(query2)
print("Anzahl von Städten mit Bevölkerungsdichte größer als 2000 in BaWü: ",mycol.count_documents(query2))

Anzahl von Städten mit Bevölkerungsdichte größer als 2000 in BaWü:  43


In [8]:
import folium

In [25]:
m = folium.Map(location=[48.74388,9.10014],zoom_start=5)

In [37]:
for citie in points:
    folium.Marker( location=[ citie['lat'], citie['lon'] ], fill_color='#43d9de', radius=8,tooltip=citie['date']).add_to(m)

In [38]:
#m

### Query 2: Gruppierung der Flächen nach Datum

In [65]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$date", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)
list_date

[{'_id': '2018-09-01', 'burned_area': 16270},
 {'_id': '2004-12-01', 'burned_area': 10843},
 {'_id': '2014-10-01', 'burned_area': 16937},
 {'_id': '2018-08-01', 'burned_area': 16926},
 {'_id': '2002-06-01', 'burned_area': 10574},
 {'_id': '2003-04-01', 'burned_area': 14209},
 {'_id': '2008-03-01', 'burned_area': 12891},
 {'_id': '2018-05-01', 'burned_area': 11664},
 {'_id': '2017-10-01', 'burned_area': 15904},
 {'_id': '2020-01-01', 'burned_area': 10310},
 {'_id': '2020-02-01', 'burned_area': 10833},
 {'_id': '2004-05-01', 'burned_area': 14329},
 {'_id': '2016-11-01', 'burned_area': 10866},
 {'_id': '2018-02-01', 'burned_area': 9921},
 {'_id': '2006-08-01', 'burned_area': 22119},
 {'_id': '2009-08-01', 'burned_area': 18618},
 {'_id': '2008-02-01', 'burned_area': 10568},
 {'_id': '2016-10-01', 'burned_area': 13229},
 {'_id': '2007-01-01', 'burned_area': 10600},
 {'_id': '2016-03-01', 'burned_area': 9851},
 {'_id': '2004-06-01', 'burned_area': 13385},
 {'_id': '2006-07-01', 'burned_area'

In [66]:
import pandas as pd
df_date = pd.DataFrame(list_date)

In [68]:
import plotly.express as px

fig = px.bar(df_date, x="_id", y="burned_area", title='Life expectancy in Canada')
fig.show()

In [57]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$country", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)
list_date

[{'_id': 'CY', 'burned_area': 100},
 {'_id': 'PR', 'burned_area': 101},
 {'_id': 'ER', 'burned_area': 1134},
 {'_id': 'TG', 'burned_area': 5824},
 {'_id': 'AM', 'burned_area': 1081},
 {'_id': 'BS', 'burned_area': 498},
 {'_id': 'PG', 'burned_area': 6594},
 {'_id': 'TN', 'burned_area': 671},
 {'_id': 'FK', 'burned_area': 63},
 {'_id': 'YE', 'burned_area': 2},
 {'_id': 'MT', 'burned_area': 1},
 {'_id': 'TM', 'burned_area': 816},
 {'_id': 'DM', 'burned_area': 1},
 {'_id': 'PE', 'burned_area': 5757},
 {'_id': 'RW', 'burned_area': 623},
 {'_id': 'ES', 'burned_area': 5621},
 {'_id': 'SV', 'burned_area': 1595},
 {'_id': 'EE', 'burned_area': 169},
 {'_id': 'AO', 'burned_area': 121719},
 {'_id': 'LR', 'burned_area': 729},
 {'_id': 'CV', 'burned_area': 1},
 {'_id': 'IE', 'burned_area': 231},
 {'_id': 'DJ', 'burned_area': 8},
 {'_id': 'LT', 'burned_area': 624},
 {'_id': 'CN', 'burned_area': 56139},
 {'_id': 'DZ', 'burned_area': 5049},
 {'_id': 'GL', 'burned_area': 35},
 {'_id': 'FR', 'burned_area

In [63]:
df_date = pd.DataFrame(list_date).sort_values(by=['burned_area'], ascending= False)
df_date

,_id,burned_area
48,RU,379365
95,BR,297973
40,AU,231628
103,CD,144610
91,ZM,135869
...,...,...
118,YT,1
37,VI,1
175,CW,1
20,CV,1


In [64]:
fig = px.bar(df_date, x="_id", y="burned_area", title='Life expectancy in Canada')
fig.show()

## Verbrannte Fläche pro Vegetationstyp

In [ ]:
# Summe der Verbranten Fläche ->  pro Vegetatoionstyp
# Waterfall Diagramm

In [4]:

list_veg = list ()
agg_result= mycol.aggregate([{ "$group":
  { "_id" : None,  
    "burned_area"  : { "$sum": "$burned_area" }, 
    "no_data_burned_area"  : { "$sum":"$no_data_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_needleleaved_evergreen_burned_area"  : { "$sum":"$tree_cover_needleleaved_evergreen_burned_area" },
    "tree_cover_needleleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_needleleaved_deciduous_burned_area" },
    "tree_cover_mixed_leave_burned_area"  : { "$sum":"$tree_cover_mixed_leave_burned_area"},
    "cropland_burned_area"  : { "$sum":"$cropland_burned_area" },
    "mosaic_tree_grass_burned_area"  : { "$sum":"$mosaic_tree_grass_burned_area" },
    "flooded_area_burned_area"  : { "$sum":"$flooded_area_burned_area" },
    "other_vegetation_burned_area"  : { "$sum":"$other_vegetation_burned_area" },}
}])
for i in agg_result:
    list_veg.append(i)
list_veg

[{'_id': None,
  'burned_area': 89940704892432.0,
  'no_data_burned_area': 7004942864842.0,
  'tree_cover_broadleaved_deciduous_burned_area': 35183926581228.0,
  'tree_cover_needleleaved_evergreen_burned_area': 807798707721.0,
  'tree_cover_needleleaved_deciduous_burned_area': 988045537799.0,
  'tree_cover_mixed_leave_burned_area': 151480670524.0,
  'cropland_burned_area': 5687008018231.0,
  'mosaic_tree_grass_burned_area': 4188674581006.0,
  'flooded_area_burned_area': 2544568553482.0,
  'other_vegetation_burned_area': 31979288438507.0}]

In [5]:
df_veg = pd.DataFrame(list_veg)

In [6]:
df_veg= df_veg.drop(columns="_id")

In [48]:
df_veg_trans = df_veg.transpose()

In [45]:
fig = px.bar(df_veg, title='Life expectancy in Canada')
fig.show()

In [7]:
df_veg

,burned_area,no_data_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,flooded_area_burned_area,other_vegetation_burned_area
0,8.994070e+13,7.004943e+12,3.518393e+13,8.077987e+11,9.880455e+11,1.514807e+11,5.687008e+12,4.188675e+12,2.544569e+12,3.197929e+13


In [20]:
import plotly.graph_objects as go

fig1  = go.Figure()
hrz = ["burned_area", "no_data_burned_area","tree_cover_broadleaved_deciduous_burned_area",
       "tree_cover_needleleaved_evergreen_burned_area","tree_cover_needleleaved_deciduous_burned_area", "cropland_burned_area", "mosaic_tree_grass_burned_area", "flooded_area_burned_area", "other_vegetation_burned_area"
       "Final"]
vrt = [ df_veg["burned_area"][0], 
       -df_veg["no_data_burned_area"][0],
       -df_veg["tree_cover_broadleaved_deciduous_burned_area"][0],
       -df_veg["tree_cover_needleleaved_evergreen_burned_area"][0],
       -df_veg["tree_cover_needleleaved_deciduous_burned_area"][0],
       -df_veg["cropland_burned_area"][0],
       -df_veg["mosaic_tree_grass_burned_area"][0],
       -df_veg["flooded_area_burned_area"][0],
       -df_veg["other_vegetation_burned_area"][0],
       None]
fig1.add_trace(go.Waterfall(                 
                  x = hrz, y = vrt,
                  base = 0,
                  measure = [ "absolute","relative",    
                              "relative","relative",
                              "relative","total" ]                        
                ))                                 
fig1.show()



In [ ]:
# Abfrage EU Länder
# Umwandlung in Dataframe

In [84]:
df_countries = pd.read_csv('C:\dev\eda_esa_fire\doc\country_codes.csv', on_bad_lines='skip', delimiter=',')

In [91]:
europe = df_countries[df_countries['region'] == 'Europe']['alpha-2'].to_list()
europe

['AX',
 'AL',
 'AD',
 'AT',
 'BY',
 'BE',
 'BA',
 'BG',
 'HR',
 'CZ',
 'DK',
 'EE',
 'FO',
 'FI',
 'FR',
 'DE',
 'GI',
 'GR',
 'GG',
 'VA',
 'HU',
 'IS',
 'IE',
 'IM',
 'IT',
 'JE',
 'LV',
 'LI',
 'LT',
 'LU',
 'MT',
 'MC',
 'ME',
 'NL',
 'MK',
 'NO',
 'PL',
 'PT',
 'RO',
 'RU',
 'SM',
 'RS',
 'SK',
 'SI',
 'ES',
 'SJ',
 'SE',
 'CH',
 'UA',
 'GB']

In [ ]:
#list(db.users.find({"document_up.tags":{"$in":["solide"]}}))